In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore',category=DeprecationWarning)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

In [2]:
df_temp = pd.read_csv("../meteo_viseu_pinhao_aveiro/dados_temperatura.csv")
df_prec = pd.read_csv("../meteo_viseu_pinhao_aveiro/dados_precipitacao.csv")
df_energy = pd.read_csv("output.csv")

In [3]:
temp_cols = df_temp.columns
prec_cols = df_prec.columns

## IPMA Temperature

In [4]:
df_temp

,Time,Temp_Viseu,Temp_Pinhao,Temp_Aveiro
0,2020-10-01 00:00:00,11.1,13.2,16.8
1,2020-10-01 00:10:00,11.1,13.2,16.7
2,2020-10-01 00:20:00,10.9,13.2,16.6
3,2020-10-01 00:30:00,11.3,13.3,16.5
4,2020-10-01 00:40:00,11.5,13.4,16.4
...,...,...,...,...
131323,2023-03-31 23:10:00,-990.0,-990.0,-990.0
131324,2023-03-31 23:20:00,-990.0,-990.0,-990.0
131325,2023-03-31 23:30:00,-990.0,-990.0,-990.0
131326,2023-03-31 23:40:00,-990.0,-990.0,-990.0


## IPMA Precipitation

In [5]:
df_prec

,Time,Precip_Viseu,Precip_Pinhao,Precip_Aveiro
0,2020-10-01 00:00:00,0.0,0.0,0.0
1,2020-10-01 00:10:00,0.0,0.0,0.0
2,2020-10-01 00:20:00,0.0,0.0,0.0
3,2020-10-01 00:30:00,0.0,0.0,0.0
4,2020-10-01 00:40:00,0.0,0.0,0.0
...,...,...,...,...
131323,2023-03-31 23:10:00,-990.0,-990.0,-990.0
131324,2023-03-31 23:20:00,-990.0,-990.0,-990.0
131325,2023-03-31 23:30:00,-990.0,-990.0,-990.0
131326,2023-03-31 23:40:00,-990.0,-990.0,-990.0


## Created Energy Dataset

In [6]:
df_energy

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35027,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35028,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35029,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35030,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


## Preprocessing

In [7]:
print("NaN values:\n",df_temp.isna().sum())
print("Negative values:",(df_temp.values[:,1:] < 0).sum())

NaN values:
 Time           0
Temp_Viseu     0
Temp_Pinhao    0
Temp_Aveiro    0
dtype: int64
Negative values: 41091


In [8]:
print("NaN values:\n",df_prec.isna().sum())
print("Negative values:",(df_prec.values[:,1:] < 0).sum())

NaN values:
 Time             0
Precip_Viseu     0
Precip_Pinhao    0
Precip_Aveiro    0
dtype: int64
Negative values: 16243


In [9]:
print("NaN values:\n",df_energy.isna().sum())
print("Negative values:",(df_energy.values[:,1:] < 0).sum())

NaN values:
 Time      0
PT41CV    0
PT44AL    0
PT15VP    0
PT69QG    0
PT43QL    0
PT17RW    0
PT87ZW    0
dtype: int64
Negative values: 0


### Equalize intervals

In [12]:
def resample_dataset_15_minutes(df):
    df['Time'] = pd.to_datetime(df['Time'])
    start = time.time()
    df_ind = df.set_index('Time')
    df_split = np.array_split(df_ind, len(df_ind)/6)
    df_time = df_ind.resample('15T').apply(lambda x: x[0])
    df_split15 = np.array_split(df_time, len(df_time)/4)
    for hour in range(len(df_split)):
        for col in range(df_time.shape[1]):
            df_split15[hour].iloc[0,col] = df_split[hour].iloc[0,col]
            df_split15[hour].iloc[1,col] = (df_split[hour].iloc[1,col] + df_split[hour].iloc[2,col]) / 2
            df_split15[hour].iloc[2,col] = df_split[hour].iloc[3,col]
            df_split15[hour].iloc[3,col] = (df_split[hour].iloc[4,col] + df_split[hour].iloc[5,col]) / 2
    df_concat = pd.DataFrame(np.concatenate(df_split15))
    dataframe = pd.DataFrame()
    dataframe.index = pd.date_range(start='2020-10-01', end='2023-04-01', closed='left', freq='15T')
    dataframe['Time'] = dataframe.index
    dataframe = dataframe.reset_index(drop=True)
    df_resampled = pd.concat([dataframe, df_concat], axis=1)
    print('Elapsed time: %.4f seconds' % (time.time() - start))
    return df_resampled

In [11]:
df_resampled = resample_dataset_15_minutes(df_temp)
df_resampled.columns = temp_cols
df_resampled

Elapsed time: 33.1315 seconds


,Time,Temp_Viseu,Temp_Pinhao,Temp_Aveiro
0,2020-10-01 00:00:00,11.10,13.20,16.80
1,2020-10-01 00:15:00,11.00,13.20,16.65
2,2020-10-01 00:30:00,11.30,13.30,16.50
3,2020-10-01 00:45:00,11.45,13.35,16.40
4,2020-10-01 01:00:00,10.80,13.40,16.40
...,...,...,...,...
87547,2023-03-31 22:45:00,-990.00,-990.00,-990.00
87548,2023-03-31 23:00:00,-990.00,-990.00,-990.00
87549,2023-03-31 23:15:00,-990.00,-990.00,-990.00
87550,2023-03-31 23:30:00,-990.00,-990.00,-990.00


In [12]:
df_resampled.to_pickle("df_temperature.pkl")  

In [13]:
df_resampled_prec = resample_dataset_15_minutes(df_prec)
df_resampled_prec.columns = prec_cols
df_resampled_prec

Elapsed time: 34.0281 seconds


,Time,Precip_Viseu,Precip_Pinhao,Precip_Aveiro
0,2020-10-01 00:00:00,0.0,0.0,0.0
1,2020-10-01 00:15:00,0.0,0.0,0.0
2,2020-10-01 00:30:00,0.0,0.0,0.0
3,2020-10-01 00:45:00,0.0,0.0,0.0
4,2020-10-01 01:00:00,0.0,0.0,0.0
...,...,...,...,...
87547,2023-03-31 22:45:00,-990.0,-990.0,-990.0
87548,2023-03-31 23:00:00,-990.0,-990.0,-990.0
87549,2023-03-31 23:15:00,-990.0,-990.0,-990.0
87550,2023-03-31 23:30:00,-990.0,-990.0,-990.0


## Data with more metrics

In [31]:
station_viseu = pd.read_csv('../meteo_viseu_pinhao_aveiro/dados_01200560.csv')
station_aveiro = pd.read_csv('../meteo_viseu_pinhao_aveiro/dados_01210702.csv')
station_viseu_cols = station_viseu.columns
station_aveiro_cols = station_aveiro.columns

In [32]:
station_viseu

,Station,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,1200560,2020-10-1 0:0:00,11.1,11.2,11.0,285.0,287.0,3.1,4.4,0.0,-990.0
1,1200560,2020-10-1 0:10:00,11.1,11.2,10.9,292.0,293.0,3.0,4.0,0.0,-990.0
2,1200560,2020-10-1 0:20:00,10.9,11.1,10.8,299.0,304.0,2.7,3.7,0.0,-990.0
3,1200560,2020-10-1 0:30:00,11.3,11.5,11.0,260.0,276.0,2.0,3.0,0.0,-990.0
4,1200560,2020-10-1 0:40:00,11.5,11.6,11.4,215.0,214.0,2.0,2.6,0.0,-990.0
...,...,...,...,...,...,...,...,...,...,...,...
131323,1200560,2023-3-31 23:10:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
131324,1200560,2023-3-31 23:20:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
131325,1200560,2023-3-31 23:30:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
131326,1200560,2023-3-31 23:40:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0


In [33]:
station_aveiro

,Station,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,1210702,2020-10-1 0:0:00,16.8,16.9,16.6,167.0,179.0,1.4,2.4,0.0,0.0
1,1210702,2020-10-1 0:10:00,16.7,16.9,16.5,169.0,163.0,1.7,2.5,0.0,0.0
2,1210702,2020-10-1 0:20:00,16.6,16.8,16.5,175.0,168.0,1.4,2.4,0.0,0.0
3,1210702,2020-10-1 0:30:00,16.5,16.6,16.4,171.0,175.0,1.2,2.3,0.0,0.0
4,1210702,2020-10-1 0:40:00,16.4,16.6,16.3,172.0,182.0,1.1,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
131323,1210702,2023-3-31 23:10:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
131324,1210702,2023-3-31 23:20:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
131325,1210702,2023-3-31 23:30:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
131326,1210702,2023-3-31 23:40:00,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0


In [34]:
print("Station Viseu")
print("NaN values:",station_viseu.isna().sum().sum())
print("Negative values:",(station_viseu.values[:,2:] < 0).sum())
print("\nStation Aveiro")
print("NaN values:",station_aveiro.isna().sum().sum())
print("Negative values:",(station_aveiro.values[:,2:] < 0).sum())

Station Viseu
NaN values: 0
Negative values: 147840

Station Aveiro
NaN values: 0
Negative values: 18041


In [35]:
station_viseu_resample = resample_dataset_15_minutes(station_viseu)
station_viseu_resample.drop(0, inplace=True, axis=1)
station_viseu_resample.columns = station_viseu_cols[1:]
station_viseu_resample

Elapsed time: 136.4784 seconds


,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,2020-10-01 00:00:00,11.10,11.20,11.00,285.0,287.0,3.10,4.40,0.0,-990.0
1,2020-10-01 00:15:00,11.00,11.15,10.85,295.5,298.5,2.85,3.85,0.0,-990.0
2,2020-10-01 00:30:00,11.30,11.50,11.00,260.0,276.0,2.00,3.00,0.0,-990.0
3,2020-10-01 00:45:00,11.45,11.60,11.20,196.0,211.0,1.70,2.55,0.0,-990.0
4,2020-10-01 01:00:00,10.80,11.10,10.60,162.0,191.0,1.30,1.70,0.0,-990.0
...,...,...,...,...,...,...,...,...,...,...
87547,2023-03-31 22:45:00,-990.00,-990.00,-990.00,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87548,2023-03-31 23:00:00,-990.00,-990.00,-990.00,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87549,2023-03-31 23:15:00,-990.00,-990.00,-990.00,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87550,2023-03-31 23:30:00,-990.00,-990.00,-990.00,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0


In [36]:
station_aveiro_resample = resample_dataset_15_minutes(station_aveiro)
station_aveiro_resample.drop(0, inplace=True, axis=1)
station_aveiro_resample.columns = station_aveiro_cols[1:]
station_aveiro_resample

Elapsed time: 152.8157 seconds


,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,2020-10-01 00:00:00,16.80,16.90,16.6,167.0,179.0,1.40,2.40,0.0,0.0
1,2020-10-01 00:15:00,16.65,16.85,16.5,172.0,165.5,1.55,2.45,0.0,0.0
2,2020-10-01 00:30:00,16.50,16.60,16.4,171.0,175.0,1.20,2.30,0.0,0.0
3,2020-10-01 00:45:00,16.40,16.60,16.3,167.0,171.5,1.20,2.30,0.0,0.0
4,2020-10-01 01:00:00,16.40,16.50,16.4,163.0,172.0,1.30,2.50,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
87547,2023-03-31 22:45:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87548,2023-03-31 23:00:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87549,2023-03-31 23:15:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0
87550,2023-03-31 23:30:00,-990.00,-990.00,-990.0,-990.0,-990.0,-990.00,-990.00,-990.0,-990.0


In [38]:
df_shared = pd.read_csv('output.csv')
#df_shared.reset_index(drop=True, inplace=True)
df_shared

,Time,PT41CV,PT44AL,PT15VP,PT69QG,PT43QL,PT17RW,PT87ZW
0,2021-11-01 00:15:00,0.028,0.452,0.000,0.828,0.128,0.056,0.552
1,2021-11-01 00:30:00,0.080,0.440,0.000,0.796,0.160,0.056,0.492
2,2021-11-01 00:45:00,0.092,0.424,0.000,0.832,0.220,0.056,0.308
3,2021-11-01 01:00:00,0.040,0.408,0.000,0.800,0.404,0.088,0.156
4,2021-11-01 01:15:00,0.076,0.396,0.000,0.836,0.156,0.116,0.160
...,...,...,...,...,...,...,...,...
35027,2022-10-31 23:00:00,0.120,0.448,0.108,0.220,0.164,0.144,0.368
35028,2022-10-31 23:15:00,0.036,0.424,0.124,0.252,0.228,0.164,0.396
35029,2022-10-31 23:30:00,0.064,0.392,0.148,0.268,0.224,0.220,0.356
35030,2022-10-31 23:45:00,0.132,0.260,0.132,0.276,0.212,0.176,0.392


In [62]:
st_viseu_intersect = station_viseu_resample.iloc[38017:73057].reset_index(drop=True)
print("Nan values:", st_viseu_intersect.isna().sum().sum())
print("Negative values:",(st_viseu_intersect.values[:,1:] < 0).sum())
st_viseu_intersect

Nan values: 0
Negative values: 38649


,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,2021-11-01 00:15:00,13.10,13.15,13.00,270.0,281.5,4.35,6.60,0.05,-990.0
1,2021-11-01 00:30:00,13.00,13.10,12.90,272.0,281.0,5.10,7.50,0.00,-990.0
2,2021-11-01 00:45:00,12.95,13.05,12.80,270.5,267.5,5.60,8.25,0.10,-990.0
3,2021-11-01 01:00:00,12.80,12.90,12.70,265.0,270.0,4.90,8.00,0.10,-990.0
4,2021-11-01 01:15:00,12.55,12.80,12.45,260.0,259.0,4.05,6.55,0.05,-990.0
...,...,...,...,...,...,...,...,...,...,...
35035,2022-10-31 23:00:00,8.40,8.70,8.30,324.0,326.0,3.50,4.00,0.00,-990.0
35036,2022-10-31 23:15:00,8.10,8.40,7.90,339.0,335.0,2.25,2.95,0.05,-990.0
35037,2022-10-31 23:30:00,8.50,8.80,8.20,341.0,332.0,1.70,2.10,0.00,-990.0
35038,2022-10-31 23:45:00,8.60,8.80,8.45,356.0,354.0,2.40,2.75,0.00,-990.0


In [59]:
st_av_intersect = station_aveiro_resample.iloc[38017:73057].reset_index(drop=True)
print("Nan values:", st_av_intersect.isna().sum().sum())
print("Negative values:",(st_av_intersect.values[:,1:] < 0).sum())
st_av_intersect

Nan values: 0
Negative values: 2135


,Time,Temp_Med,Temp_Max,Temp_Min,Rumo_Vento_Med,Rumo_Vento_Max,Intensidade_Vento_Med,Intensidade_Vento_Max,Precip,Rad_Total
0,2021-11-01 00:15:00,16.50,16.60,16.40,337.5,342.0,2.20,2.90,0.05,0.0
1,2021-11-01 00:30:00,16.50,16.50,16.40,338.0,335.0,1.90,2.60,0.00,0.0
2,2021-11-01 00:45:00,16.40,16.55,16.35,316.0,321.5,1.60,2.25,0.05,0.0
3,2021-11-01 01:00:00,16.40,16.50,16.30,293.0,308.0,1.60,2.30,0.10,0.0
4,2021-11-01 01:15:00,16.30,16.40,16.25,295.5,305.5,1.90,2.65,0.05,0.0
...,...,...,...,...,...,...,...,...,...,...
35035,2022-10-31 23:00:00,13.20,13.40,13.00,153.0,109.0,0.40,1.10,0.00,0.0
35036,2022-10-31 23:15:00,13.25,13.35,13.15,153.5,120.0,1.05,2.20,0.00,0.0
35037,2022-10-31 23:30:00,13.30,13.40,13.20,122.0,101.0,0.90,1.90,0.00,0.0
35038,2022-10-31 23:45:00,13.25,13.40,13.15,115.5,119.0,1.15,2.05,0.00,0.0
